In [8]:
%load_ext autoreload
%autoreload 2

In [10]:
import sys
sys.path.append('..')

In [86]:
import os

import torch
from torchsummary import summary

In [85]:
from dataset import *
from cnn import CNNetwork

In [78]:
if torch.cuda.is_available():
        device = "cuda"
else:
        device = "cpu"
print(f"Using device {device}")

Using device cpu


In [109]:
mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=16000,
        n_fft=1024,
        hop_length=512,
        n_mels=64
    )
dataset = VoiceDataset('../data/train', mel_spectrogram, 16000, device)

In [110]:
len(dataset)

5717

In [111]:
dataset[0]

(tensor([[[0.2647, 0.0247, 0.0324,  ..., 0.0230, 0.1026, 0.5454],
          [0.0812, 0.0178, 0.0890,  ..., 0.2376, 0.5061, 0.5292],
          [0.0052, 0.0212, 0.1341,  ..., 0.9336, 0.2778, 0.1372],
          ...,
          [0.5154, 0.3950, 0.4497,  ..., 0.4916, 0.4505, 0.7709],
          [0.1919, 0.4804, 0.5144,  ..., 0.5931, 0.4466, 0.4706],
          [0.1208, 0.4357, 0.4016,  ..., 0.5168, 0.7007, 0.3696]]]),
 0)

In [112]:
dataset[0][0].shape

torch.Size([1, 64, 157])

In [144]:
cnn = CNNetwork()
# summary(cnn, (1, 64, 44))

In [114]:
torch.tensor(0)

tensor(0)

In [115]:
dataset.label_mapping

{'aman': 0, 'imran': 1, 'labib': 2}

In [116]:
dataset[0]

(tensor([[[0.2647, 0.0247, 0.0324,  ..., 0.0230, 0.1026, 0.5454],
          [0.0812, 0.0178, 0.0890,  ..., 0.2376, 0.5061, 0.5292],
          [0.0052, 0.0212, 0.1341,  ..., 0.9336, 0.2778, 0.1372],
          ...,
          [0.5154, 0.3950, 0.4497,  ..., 0.4916, 0.4505, 0.7709],
          [0.1919, 0.4804, 0.5144,  ..., 0.5931, 0.4466, 0.4706],
          [0.1208, 0.4357, 0.4016,  ..., 0.5168, 0.7007, 0.3696]]]),
 0)

In [117]:
from datetime import datetime
now = datetime.now()

In [107]:
now.strftime("%Y%m%d_%H%M%S")

'20230512_222912'

In [145]:
cnn.load_state_dict(torch.load("../models/void_20230512_225714.pth"))

<All keys matched successfully>

In [151]:
x, y = dataset[10]

In [152]:
labels = dataset._labels

In [153]:
input = x.unsqueeze_(0) 

In [182]:
def predict(model, input, target, class_mapping):
    model.eval()
    with torch.no_grad():
        predictions = model(input)
        predicted_index = predictions[0].argmax(0)
        predicted = class_mapping[predicted_index]
        expected = class_mapping[target]
    return predictions

In [155]:
predict(cnn, input, y, labels)

tensor([[1.0000e+00, 1.3728e-20, 2.8026e-44]])


('aman', 'aman')

In [156]:
input

tensor([[[[0.0259, 0.1384, 0.0784,  ..., 0.0000, 0.0000, 0.0000],
          [0.0334, 0.1320, 0.0701,  ..., 0.0000, 0.0000, 0.0000],
          [0.0481, 0.0324, 0.0545,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.2665, 0.3647, 0.3147,  ..., 0.0000, 0.0000, 0.0000],
          [0.2710, 0.3796, 0.2160,  ..., 0.0000, 0.0000, 0.0000],
          [0.1950, 0.2607, 0.1905,  ..., 0.0000, 0.0000, 0.0000]]]])

In [157]:
input.shape

torch.Size([1, 1, 64, 157])

In [158]:
x.shape

torch.Size([1, 1, 64, 157])

In [159]:
os.path.join('data', 'aman', 'aman_1')

'data/aman/aman_1'

In [175]:
test_dataset =  VoiceDataset('../data/test', mel_spectrogram, 16000, device)

In [177]:
test_dataset.audio_files_labels

[('aman_test2.m4a', 'aman'),
 ('aman_6.m4a', 'aman'),
 ('aman_4.m4a', 'aman'),
 ('aman_5.m4a', 'aman'),
 ('aman_1.m4a', 'aman'),
 ('aman_2.m4a', 'aman'),
 ('aman_3.m4a', 'aman'),
 ('aman_test.m4a', 'aman'),
 ('Imran 6.m4a', 'imran'),
 ('Imran 5.m4a', 'imran'),
 ('Imran 4.m4a', 'imran'),
 ('Imran 1.m4a', 'imran'),
 ('Imran 3.m4a', 'imran'),
 ('Imran 2.m4a', 'imran'),
 ('labib_6.m4a', 'labib'),
 ('labib_4.m4a', 'labib'),
 ('labib_5.m4a', 'labib'),
 ('labib_1.m4a', 'labib'),
 ('labib_2.m4a', 'labib'),
 ('labib_3.m4a', 'labib')]

In [178]:
test_input, test_output = test_dataset[0]
test_input.shape

torch.Size([1, 64, 157])

In [179]:
test_input = test_input.unsqueeze_(0) 

In [180]:
test_input.shape

torch.Size([1, 1, 64, 157])

In [185]:
output = predict(cnn, test_input, test_output, test_dataset._labels)

tensor([[1., 0., 0.]])


In [188]:
pred = torch.argmax(output, 1)